In [59]:
import requests

# Replace 'YOUR_API_KEY' with your actual API key

url = 'https://projects.fivethirtyeight.com/biden-approval-rating/polls.json'

# Make a GET request
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}")
    print(response.text)


In [36]:

#i used chatgbt to sort data here 

Biden_Opinion_Polls = [{'startDate': poll['startDate'], 'approve': poll['answers'][0]['pct'], 'disapprove': poll['answers'][1]['pct']} for poll in data]

    

In [51]:
import requests
import pandas as pd

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('output.csv', index=False)




In [38]:
import pandas as pd

Biden_Opinion_Polls= [{'startDate': poll['startDate'], 'approve': poll['answers'][0]['pct'], 'disapprove': poll['answers'][1]['pct']} for poll in data]

Biden_dataframe= pd.DataFrame(Biden_Opinion_Polls)

Biden_dataframe.to_csv('Biden_Opinion_Polls_df.csv', index=False)


In [39]:
pip install statsmodels


using https://edition.cnn.com/2022/12/31/us/top-100-digital-stories-2022-trnd/index.html 

https://scrippsnews.com/stories/looking-back-at-the-biggest-headlines-of-2023/

used chatgbt to help with more complicated vega altaire graph

In [56]:
events_data = pd.DataFrame({
    'Date': ['2021-06-16', '2021-03-11', '2021-11-06', '2021-08-26', '2022-03-08', '2022-08-24','2022-02-26','2023-05-11','2023-09-14','2023-11-14','2023-07-2','2023-01-07'],
    'Event': ['Geneva Summit with Putin', 'American Rescue Plan', 'Infrastructure Investment and Jobs Act Signed', 'Hurricane Ida', 'Ban on Russian energy imports', 'Student Loan forgiveness plan','Trump sides with Putin as Biden tries to stop a war','Title 42 expires as federal government tries to handle migration','Hunter Biden indicted by federal prosecutors on gun charges','Biden met with Chinese leader Xi Jinping','Interest rates jump to highest levels in 22 years','Republicans take majority in the House of Representatives for the first time in four years'],})

events_data.to_csv('event_data.csv', index=False)


In [49]:
import altair as alt
import pandas as pd
import statsmodels.api as sm

# Assume 'Biden_dataframe' is your DataFrame
df_long = pd.melt(Biden_dataframe, id_vars=['startDate'], value_vars=['approve', 'disapprove'],
                  var_name='Opinion', value_name='Percentage')

# Altair Chart for points with filled circles
chart_points = alt.Chart(df_long).mark_point(size=50, opacity=0.25, filled=True).encode(
    x=alt.X('startDate:T', axis=alt.Axis(tickCount='month', format='%b %Y', labelAngle=-90)),
    y=alt.Y('Percentage:Q', axis=alt.Axis(title='Percentage')),
    color=alt.Color('Opinion:N', scale=alt.Scale(domain=['approve', 'disapprove'], range=['green', 'red']), legend=alt.Legend(title='Opinion')),
).properties(
    title='Biden Approval Rates Overtime',
    width=600,
    height=400
)

# Calculate smoothed values using loess from statsmodels
loess_smoothed = df_long.groupby(['Opinion', 'startDate']).mean().reset_index()
loess_smoothed['smoothed'] = loess_smoothed.groupby('Opinion')['Percentage'].transform(lambda x: sm.nonparametric.lowess(x, range(len(x)), frac=0.3)[:, 1])

# Altair Chart for smoothed lines
chart_lines = alt.Chart(loess_smoothed).mark_line(color='black').encode(
    x=alt.X('startDate:T', axis=alt.Axis(tickCount='month', format='%b %Y', labelAngle=-90), title='Date'),
    y=alt.Y('smoothed:Q', axis=alt.Axis(title='Percentage')),
    detail='Opinion:N'
)


# Vertical lines as annotations for events
vertical_lines = alt.Chart(events_data).mark_rule(color='blue', strokeWidth=1.5,opacity=0.5).encode(
    x='Date:T',
    tooltip=['Event:N'],
)

# Combine scatter plot, filled circles, black smooth lines, and raw data points with vertical lines
combined_chart = alt.layer(chart_points, chart_lines, vertical_lines).interactive()

# Show the combined chart
combined_chart


alt.LayerChart(...)